# Предобработка данных

## Импорт библиотек

In [1]:
import pandas as pd
import cv2
import os
import re
import numpy as np
import datetime as dt

from PIL import Image as PILImage

from img2table.document import Image

from pytesseract import Output
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

#import pyocr
#pyocr.tesseract.TESSERACT_CMD = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
#from IPython.display import display_html
#from openpyxl import load_workbook
#from io import BytesIO
#from img2table.ocr import TesseractOCR
#TesseractOCR.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

## Папка с исходниками (для загрузки изображений по одному)

In [2]:
SOURCE_DIR = '406/'

try:
    for filename in os.listdir(SOURCE_DIR):
        if 'jpg' in filename.lower() or 'jpeg' in filename.lower() or 'png' in filename.lower():
            image_path = SOURCE_DIR + filename
        else:
            print('NO IMAGE IN DIRECTORY')
except:
    print('ENTER IMAGE NAME')
    p = input()
    image_path = f"app/405/{p}.jpg"

ENTER IMAGE NAME
1


## Функции для работы

### Препроцессинг

**Обрезка** Для этой задачи - обрезаем пополам. Потому что у справки формы 405 таблица всегда в нижней половине.

**Оттенки серого**

**Удаление шумов**

**Приведение к пороговому значению** Gaussian

In [3]:
def pre(img):
    height = int(img.shape[0])
    half = int(img.shape[0] / 2)
    cut = img[half:height, :]
    gray = cv2.cvtColor(cut, cv2.COLOR_BGR2GRAY)
    noise = cv2.medianBlur(gray,1)
    thresh = cv2.adaptiveThreshold(noise,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,11)
    out = thresh
    return out

#ret,thresh3 = cv.threshold(img,127,255,cv.THRESH_TRUNC)
#ret,thresh4 = cv.threshold(img,127,255,cv.THRESH_TOZERO)

### Поиск координат ячеек таблиц с помощью img2table

In [4]:
def search(path):
    img = Image(src=path)
    extracted_tables = img.extract_tables()
    table_img = (cv2.imread(path))
    find_cell = []
    for table in extracted_tables:
        for row in table.content.values():
            for cell in row:
                cv2.rectangle(table_img, (cell.bbox.x1, cell.bbox.y1), (cell.bbox.x2, cell.bbox.y2), (255, 0, 0), 2)
                rect = [cell.bbox.y1, cell.bbox.y2, cell.bbox.x1, cell.bbox.x2]
                find_cell.append(rect)
    if len(find_cell) < 5:
        return 'PREPROCESSING ERROR TRY ANOTHER IMAGE'
    else:
        return find_cell
    #PILImage.fromarray(table_img)

### Распознавание числовых значений и дат

In [5]:
def numbers(cells, path):
    cfg = r'--oem 3 --psm 7 outputbase digits'
    nums = []
    img = (cv2.imread(path))
    for place in cells:
        img_part = img[place[0]:place[1],place[2]:place[3]]
        nums.append(pytesseract.image_to_string(img_part, lang='rus', config=cfg))
    return nums

### Распознавание всего остального

In [6]:
def words(cells, path):
    cfg = r'--oem 3 --psm 7'
    wds = []
    img = (cv2.imread(path))
    for place in cells:
        img_part = img[place[0]:place[1],place[2]:place[3]]
        wds.append(pytesseract.image_to_string(img_part, lang='rus', config=cfg))
    return wds

### Объединение распознанных данных

In [7]:
def join_data(n, w):
    for i in range(len(n)):
        if n[i] == '':
            n[i] = w[i]
    return n

### Очистка выходных значений: оставляем только данные в формате "дата - тип донации - объем", удаляем пробельные символы и точки из конца строки

In [8]:
def clean(list):
    stop_head = 0
    for i in range(len(list)):
        if re.search(r'\d{2}.\d{2}.\d{4}', list[i]):
            stop_head = i
            break
    cut_head = list[stop_head:]
    for j in range(len(cut_head)-1, 0, -1):
        if re.search(r'\d{2,}', cut_head[j]):
            stop_tail = j
            break
    cut_tail = cut_head[:j+1]
    clean_list = cut_tail
    clean_counter = 0
    for k in range(len(clean_list)):
        if (not (re.search(r'\d{2}.\d{2}.\d{4}', clean_list[k]) or \
        re.search(r'\d{2,}', clean_list[k]) or \
        re.fullmatch(r'\w{,2}\s', clean_list[k]))):
            clean_list[k] = 'unknown'
            clean_counter += 1
        clean_list[k] = re.sub(r'\s', '', clean_list[k])
        clean_list[k] = re.sub(r'[-, =]', '', clean_list[k])
        if clean_list[k][-1] == '.':
            clean_list[k] = clean_list[k][:-1]
        if clean_list[k][0] == '.':
            clean_list[k] = clean_list[k][1:]
    return clean_list

### Формирование таблицы из распознанных значений и сортировка их по дате

In [9]:
def table(list):
    date = []
    volume = []
    don = []
    category = []
    error = []
    rec_counter = 0
    for i in range(len(list)):
        if re.fullmatch(r'\d{2}.\d{2}.\d{4}', list[i]):
            try:
                d = dt.datetime.strptime(list[i], '%d.%m.%Y')
                d = dt.datetime.date(d)
                rec_counter += 1
            except:
                d = None
            date.append(d)
        elif re.fullmatch(r'\d{3,4}', list[i]) and (50 <= int(list[i]) <= 750):
            volume.append(int(list[i]))
        elif re.search(r'[8, б, в, 6]', list[i]) and len(list[i]) < 3:
            don.append('Безвозмездно')
        elif re.search(r'[к, К, р, Р]', list[i]):
            category.append('Цельная кровь')
        elif re.search(r'[п, П, л, Л]', list[i]):
            category.append('Плазма')
        elif re.search(r'[т, Т, ф, Ф]', list[i]):
            category.append('Тромбоциты')
        else:
            error.append(list[i])
            volume.append(0)
            category.append('unknown')
            don.append('unknown')
    if rec_counter == 0:
        return 'RECOGNITION ERROR TRY ANOTHER IMAGE'
    else:
        result = pd.concat([pd.Series(date), pd.Series(volume), pd.Series(don), pd.Series(category)], axis=1)
        result = result.rename(columns={0: "Дата донации", 1: "Объем, мл", 2: "Тип донации", 3: "Класс крови"})
        result = result.dropna(subset=["Дата донации"]).reset_index(drop=True)
        result = result.fillna(0)
        result = result.drop_duplicates(subset=["Дата донации"]).reset_index(drop=True)
        result = result.sort_values(by='Дата донации')  
        return result

# Распознавание изображений

## Загрузка и предобработка изображений

## Распознавание

In [10]:
os.makedirs('recognized/', exist_ok=True)

input_image = cv2.imread(image_path)
preprocessed_image = pre(input_image)
pre_path = 'recognized/' + 'pre_image.png'
cv2.imwrite(pre_path, preprocessed_image)

cell = search(pre_path)
if type(cell) == str:
    print(pth, '---',cell)
    exit()
n = numbers(cell, pre_path)
w = words(cell, pre_path)
raw_out = (join_data(n, w))
cleaned_out = clean(raw_out)
out_data = table(cleaned_out)
if type(out_data) == str:
    print(pth, '---', out_data)
    exit()
name = p + '.csv'
out_data.to_csv('recognized/' + name)
print(image_path, '---', 'IMAGE RECOGNIZED')

app/405/1.jpg --- IMAGE RECOGNIZED
